# 1 Data Generation

In [ ]:
import pandas as pd
import numpy as np


# store kitchen data into dataframe
data_path = "./amazon_reviews_us_Kitchen_v1_00.tsv"
df_ori = pd.read_csv(data_path, sep='\t', usecols=[7, 12, 13])

df_ori.replace(np.inf, np.nan)
df_ori = df_ori.dropna(axis=0, how="any")
df_ori["review"] = df_ori["review_headline"] + " " + df_ori["review_body"]
df_ori.drop(columns=["review_headline", "review_body"])

,star_rating,review
0,5.0,Beautiful. Looks great on counter Beautiful. ...
1,5.0,Awesome & Self-ness I personally have 5 days s...
2,5.0,Fabulous and worth every penny Fabulous and wo...
3,5.0,Five Stars A must if you love garlic on tomato...
4,5.0,Better than sex Worth every penny! Buy one now...
...,...,...
4875083,4.0,Not as sturdy as you'd think. After a month of...
4875084,5.0,A Sweetheart of A Pan I've used my Le Creuset ...
4875085,4.0,"Ice Cream Like a Dream According to my wife, t..."
4875086,4.0,Opens anything and everything Hoffritz has a n...


In [ ]:
# extract reviews every label 50000

df_r5 = df_ori[df_ori["star_rating"] == 5.0]  
df_r4 = df_ori[df_ori["star_rating"] == 4.0] 
df_r3 = df_ori[df_ori["star_rating"] == 3.0]  
df_r2 = df_ori[df_ori["star_rating"] == 2.0]  
df_r1 = df_ori[df_ori["star_rating"] == 1.0]  

df_r5s = df_r5.sample(n=50000)
df_r4s = df_r4.sample(n=50000)
df_r3s = df_r3.sample(n=50000)
df_r2s = df_r2.sample(n=50000)
df_r1s = df_r1.sample(n=50000)
df_sp = pd.concat([df_r5s, df_r4s, df_r3s, df_r2s, df_r1s])

label = []

for star in df_sp["star_rating"]:
  if star > 3.0:
      label.append(0)
  elif star < 3.0:
      label.append(1)
  else:
      label.append(2)

df_sp["label"] = label
col_order = ["review", "label"]
df_sp = df_sp[col_order]




In [ ]:
# save sampled data if need
# output_path = "./data/dataset_sampled.csv"
# df_sp.to_csv(output_path)


In [6]:
import pandas as pd
import numpy as np

# load sampled data, avoid sampling again, if need
# file_path = "./data/dataset_sampled.csv"
# df_sp = pd.read_csv(file_path)

,Unnamed: 0,review,label
0,898464,Five Stars Thanks!,0
1,3985118,Just what I wanted I love the way chefs crush ...,0
2,2412310,So easy Very happy with this product! Bought a...,0
3,1287078,Five Stars works very well and well made,0
4,1637732,This is a great tea infuser! This is a great t...,0
...,...,...,...
249995,2809365,Not Happy with it I used it about 5 times and ...,1
249996,4512646,"Great Coffee Maker, Sneaky Company I loved thi...",1
249997,2835983,Worse than a blender I bought this because I n...,1
249998,4563097,Cuisinart? Never Again [[ASIN:B0000A1ZN7 Cuisi...,1


# 2 Word Embedding

## Import Word2Vec Model

In [ ]:
import gensim.downloader as api

# download w2v model
wv = api.load('word2vec-google-news-300')

In [4]:
import gensim

# load w2v model, if need
# wv_path = "./model/word2vec-google-news-300.gz"
# wv = gensim.models.KeyedVectors.load_word2vec_format(wv_path, binary=True)

In [5]:
# test w2v model
print(wv.similarity("excellent", "outstanding"))
print(wv.similarity("car", "vehicle"))

0.55674857
0.7821097


In [ ]:
# train the model using our own data
from gensim.models import Word2Vec

raw_sent = list(df_sp["review"])

from gensim import utils

class MyCorpus:
    def __init__(self, review):
        self.review = review

    def __iter__(self):
        for sent in self.review:
            yield utils.simple_preprocess(sent)

sentences = MyCorpus(raw_sent)
wvmodel = Word2Vec(sentences=sentences, vector_size=300, window=11, min_count=10, workers = 4)

# save the model, avoiding training again
wvwmodel.save("word2vec.model")

In [11]:
import gensim

# load the model if need
# wvmodel = gensim.models.Word2Vec.load("./model/word2vec.model")

# test the new model
print(wvmodel.wv.similarity("excellent", "outstanding"))
print(wvmodel.wv.similarity("car", "vehicle"))

0.7475236
0.68691987


## Data preprocessing and cleaning

In [7]:
from nltk.corpus import stopwords
import re
import contractions

from bs4 import BeautifulSoup


# do the same data preprocessing and cleaning as HW1
def remove_non_alphabetic(text):
    pattern = re.compile('[^a-zA-Z]')
    return pattern.sub(' ', text)

def remove_url(text):
    pattern = '[http|https]*://[a-zA-Z0-9.?#/&-_=:]*'
    url_pattern = re.compile(pattern)
    return url_pattern.sub('', text)

def remove_html(text):
    return BeautifulSoup(text).get_text()

def remove_space(text):
    space_pattern = re.compile('\s+')
    text = re.sub(space_pattern, ' ', text)
    return text

def to_lowercase(text):
    return text.lower()

def expand_contractions(text):
    return contractions.fix(text)

df_sp["review"] = df_sp["review"].apply(lambda x: to_lowercase(x))
df_sp["review"] = df_sp["review"].apply(lambda x: remove_url(x))
df_sp["review"] = df_sp["review"].apply(lambda x: remove_html(x))
df_sp["review"] = df_sp["review"].apply(lambda x: expand_contractions(x))
df_sp["review"] = df_sp["review"].apply(lambda x: remove_non_alphabetic(x))
df_sp["review"] = df_sp["review"].apply(lambda x: remove_space(x))

df_sp

,Unnamed: 0,review,label
0,898464,five stars thanks,0
1,3985118,just what i wanted i love the way chefs crush ...,0
2,2412310,so easy very happy with this product bought a ...,0
3,1287078,five stars works very well and well made,0
4,1637732,this is a great tea infuser this is a great te...,0
...,...,...,...
249995,2809365,not happy with it i used it about times and it...,1
249996,4512646,great coffee maker sneaky company i loved this...,1
249997,2835983,worse than a blender i bought this because i n...,1
249998,4563097,cuisinart never again asin b a zn cuisinart dc...,1


In [ ]:
# data preprocessing
import nltk


from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
df_cd = df_sp

stopwords = set(stopwords.words('english'))
lem_words = []
wnl = WordNetLemmatizer()

df_cd["review"] = df_cd["review"].apply(lambda x: word_tokenize(x))
df_cd["review"] = df_cd["review"].apply(lambda x: " ".join([word for word in x if word not in stopwords]))

df_cd["review"] = df_cd["review"].apply(lambda x: word_tokenize(x))
df_cd["review"] = df_cd["review"].apply(lambda x: " ".join([wnl.lemmatize(word) for word in x]))

In [ ]:
# load cleaned data (might have nan value, aovid to load directly)
# file_path = "./data/dataset_cleaned.csv"
# df_cd = pd.read_csv(file_path)

In [8]:
import pandas as pd
import numpy as np

# take data for binary classification and ternery

df_pos = df_cd[df_cd["label"] == 0]  # positive review
df_neg = df_cd[df_cd["label"] == 1]  # negative review


# split the data for binary and ternary cases
df_bin = pd.concat([df_pos, df_neg])
df_neu = df_cd[df_cd["label"] == 2] 



## Convert reviews into vectors

### Pretrained w2v model

In [ ]:
import numpy as np
# get word vector in review

pre_vec_bin = []
pre_vec_neu = []

# Average sentence vector
for df in df_bin, df_neu:
    for index, row in df.iterrows():
        review = row["review"]
        wordcount = 0
        vec = np.zeros(300)
        for word in review.split(" "):
            try:
                vec += wv[word]
                wordcount += 1
            except KeyError as e:  # when there's no such word, ignore it
                continue
        if wordcount != 0:
            vec /= wordcount
        if df.shape[0] == 200000:
            pre_vec_bin.append(vec)
        elif df.shape[0] == 50000:
            pre_vec_neu.append(vec)

pre_vec_all = pre_vec_bin + pre_vec_neu

# 10 word vectors
pre_vec10_bin = []
pre_vec10_neu = []
vec_zero = np.zeros(300)

for df in df_bin, df_neu:
    for index, row in df.iterrows():
        review = row["review"]
        split_review = review.split(" ")
        len_review = len(split_review)
        try:
            vec = wv[split_review[0]]
        except:
            vec = np.zeros(300)
        for word in split_review[1:10]:
            try:
                vec = np.append(vec, wvmodel.wv[word])
            except KeyError as e:
                continue    
        if len(vec) < 3000:
            vec_zero = np.zeros(3000 - len(vec))
            vec = np.append(vec, vec_zero)
#         print(len(vec))
        if df.shape[0] == 200000:
            pre_vec10_bin.append(vec)
        else:
            pre_vec10_neu.append(vec)
        
pre_vec10_all = pre_vec10_bin + pre_vec10_neu

In [ ]:
# save pretrained vector, if need

X_pre_bin = np.array(pre_vec_bin)
# np.save("./pretrained/xprebin_2.npy", X_pre_bin)
X_pre_all = np.array(pre_vec_all)
# np.save("./pretrained/xpreall2_2.npy", X_pre_all)

X_pre_bin10 = np.array(pre_vec10_bin)
# np.save("./pretrained/xprebin10_2.npy", X_pre_bin10)
X_pre_all10 = np.array(pre_vec10_all)
# np.save("./pretrained/xpreall10_2.npy", X_pre_all10)


In [ ]:
# df_bin["label"]

y_bin = df_bin["label"]
y_bin = np.array(y_bin.values)
# np.save('./pretrained/ybin.npy', y_bin)

y_neu = df_neu["label"]
y_neu = np.array(y_neu.values)

y_all = np.append(y_bin, y_neu)
# np.save('./pretrained/yall.npy', y_all)

### Own w2v model

In [ ]:
import numpy as np
# get word vector in review

sent_vec_bin = []
sent_vec_neu = []


# Average sentence vector
for df in df_bin, df_neu:
    for index, row in df.iterrows():
        review = row["review"]
        wordcount = 0
        vec = np.zeros(300)
        for word in review.split(" "):
            try:
                vec += wvmodel.wv[word]
                wordcount += 1
            except KeyError as e:
                continue
        if wordcount != 0:
            vec /= wordcount
        if df.shape[0] == 200000:
            sent_vec_bin.append(vec)
        elif df.shape[0] == 50000:
            sent_vec_neu.append(vec)

            
sent_vec_all = sent_vec_bin + sent_vec_neu

# 10 word vectors
sent_vec10_bin = []
sent_vec10_neu = []
vec_zero = np.zeros(300)

for df in df_bin, df_neu:
    for index, row in df.iterrows():
        review = row["review"]
        split_review = review.split(" ")
        len_review = len(split_review)
        try:
            vec = wvmodel.wv[split_review[0]]
        except:
            vec = np.zeros(300)
        for word in split_review[1:10]:
            try:
                vec = np.append(vec, wvmodel.wv[word])
            except KeyError as e:
                continue    
        if len(vec) < 3000:
            vec_zero = np.zeros(3000 - len(vec))
            vec = np.append(vec, vec_zero)
#         print(len(vec))
        if df.shape[0] == 200000:
            sent_vec10_bin.append(vec)
        else:
            sent_vec10_neu.append(vec)
        
sent_vec10_all = sent_vec10_bin + sent_vec10_neu

In [ ]:
# save vectors, avoid converting again
# own model
# average vectors 
X_bin = np.array(sent_vec_bin)
# np.save('./own/xbin.npy',X_bin)
X_all = np.array(sent_vec_all)
# np.save('./own/xall.npy',X_all)

# 10 word vectors
X10_bin = np.array(sent_vec10_bin)
# np.save('./own/x10bin.npy',X10_bin)

X10_all = np.array(sent_vec10_all)
# np.save('./own/x10all.npy',X10_all)


(200000, 3000)
(250000, 3000)


# 3 Simple Model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np

## SVM 




### Pretrained w2v model--m1

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

X = X_pre_bin
y = y_bin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm = LinearSVC(max_iter=500)
svm.fit(X_train, y_train)
pred_train = svm.predict(X_train)
print("Train")
print("Accuracy: " + str(accuracy_score(pred_train, y_train)))
print("Precision: " + str(precision_score(pred_train, y_train)))
print("Recall: " + str(recall_score(pred_train, y_train)))
print("F1: " + str(f1_score(pred_train, y_train)))

pred_test = svm.predict(X_test)
print("Test")
print("Accuracy: " + str(accuracy_score(pred_test, y_test)))
print("Precision: " + str(precision_score(pred_test, y_test)))
print("Recall: " + str(recall_score(pred_test, y_test)))
print("F1: " + str(f1_score(pred_test, y_test)))

Train
Accuracy: 0.84559375
Precision: 0.8616378933156651
Recall: 0.8348271601947628
F1: 0.8480206699271016
Test
Accuracy: 0.847225
Precision: 0.8621482481131604
Recall: 0.8372488107950684
F1: 0.8495161171168952


### Own w2v model--m2


In [ ]:
X = X_bin
y = y_bin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm = LinearSVC(max_iter=500)
svm.fit(X_train, y_train)
pred_train = svm.predict(X_train)
print("Train")
print("Accuracy: " + str(accuracy_score(pred_train, y_train)))
print("Precision: " + str(precision_score(pred_train, y_train)))
print("Recall: " + str(recall_score(pred_train, y_train)))
print("F1: " + str(f1_score(pred_train, y_train)))

pred_test = svm.predict(X_test)
print("Test")
print("Accuracy: " + str(accuracy_score(pred_test, y_test)))
print("Precision: " + str(precision_score(pred_test, y_test)))
print("Recall: " + str(recall_score(pred_test, y_test)))
print("F1: " + str(f1_score(pred_test, y_test)))

/usr/local/lib/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Train
Accuracy: 0.88163125
Precision: 0.887077619291688
Recall: 0.8775010511216086
F1: 0.8822633486469517
Test
Accuracy: 0.883275
Precision: 0.8877392912480632
Recall: 0.8799544193420531
F1: 0.8838297131198527


## Perceptron

### Pretrained w2v model--m3

In [ ]:
from sklearn.linear_model import Perceptron

X = X_pre_bin
y = y_bin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = Perceptron()
clf.fit(X_train, y_train)
pred_train = clf.predict(X_train)
print("Train")
print("Accuracy: " + str(accuracy_score(pred_train, y_train)))
print("Precision: " + str(precision_score(pred_train, y_train)))
print("Recall: " + str(recall_score(pred_train, y_train)))
print("F1: " + str(f1_score(pred_train, y_train)))

pred_test = clf.predict(X_test)
print("Test")
print("Accuracy: " + str(accuracy_score(pred_test, y_test)))
print("Precision: " + str(precision_score(pred_test, y_test)))
print("Recall: " + str(recall_score(pred_test, y_test)))
print("F1: " + str(f1_score(pred_test, y_test)))

Train
Accuracy: 0.81939375
Precision: 0.894053229657595
Recall: 0.7778768762236241
F1: 0.831928716418605
Test
Accuracy: 0.821675
Precision: 0.8927375418603489
Recall: 0.7817314425770309
F1: 0.8335550110838876


### Own w2v model--m4

In [ ]:
from sklearn.linear_model import Perceptron

X = X_bin
y = y_bin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = Perceptron()
clf.fit(X_train, y_train)
pred_train = clf.predict(X_train)
print("Train")
print("Accuracy: " + str(accuracy_score(pred_train, y_train)))
print("Precision: " + str(precision_score(pred_train, y_train)))
print("Recall: " + str(recall_score(pred_train, y_train)))
print("F1: " + str(f1_score(pred_train, y_train)))

pred_test = clf.predict(X_test)
print("Test")
print("Accuracy: " + str(accuracy_score(pred_test, y_test)))
print("Precision: " + str(precision_score(pred_test, y_test)))
print("Recall: " + str(recall_score(pred_test, y_test)))
print("F1: " + str(f1_score(pred_test, y_test)))

Train
Accuracy: 0.83746875
Precision: 0.7855062317952821
Recall: 0.8765798945341927
F1: 0.8285478819845064
Test
Accuracy: 0.840525
Precision: 0.7877242964962263
Recall: 0.8808405991504583
F1: 0.8316842133037811


# 4 Feedfoward Neueral Network

## Pretrained w2v model

### Average feature

#### Binary case--m5

In [ ]:
# load dataset if need

# X_pre_bin = np.load("./pretrained/xprebin.npy")
# y_bin = np.load("./pretrained/ybin.npy")

In [ ]:
# MLP Binary model

import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        hidden_1 = 100
        hidden_2 = 50
        hidden_3 = 2
        self.fc1 = nn.Linear(300, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return x
    
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y

    
def report_acc():
    total = 0
    correct = 0
    with torch.no_grad():
        for data, label in test_generator:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print('Accuracy on the test set: %.6s' % (correct / total))
    
    
X = torch.from_numpy(X_pre_bin).float()
y = torch.from_numpy(y_bin).float()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
params = {'batch_size':64, 'num_workers': 0, 'shuffle': True}


train_set = Dataset(X_train, y_train)

train_generator = data.DataLoader(train_set, **params)

test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)
    
model = Net()
print(model)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

    
epoch = 5

for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        optimizer.zero_grad() 
        y_pred = model(data)
        loss = criterion(y_pred, label.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss))
    
report_acc()


Net(
  (fc1): Linear(in_features=300, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Epoch:1 Training Loss: 0.561819
Epoch:2 Training Loss: 0.379455
Epoch:3 Training Loss: 0.359355
Epoch:4 Training Loss: 0.346756
Epoch:5 Training Loss: 0.337668
Accuracy on the test set: 0.8408


#### Ternary case--m6

In [ ]:
# load dataset

# X_pre_all = np.load("./pretrained/xpreall.npy")
# y_all = np.load("./pretrained/yall.npy")

In [ ]:
# MLP ternary model
import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        hidden_1 = 100
        hidden_2 = 50
        hidden_3 = 3 # for ternery
        self.fc1 = nn.Linear(300, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return x
    
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y

X = torch.from_numpy(X_pre_all).float()
y = torch.from_numpy(y_all).float()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
params = {'batch_size':64, 'num_workers': 0, 'shuffle': True}
train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, **params)
test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)
model = Net()
print(model)
# torch.set_default_tensor_type(torch.DoubleTensor)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

def report_acc():
    total = 0
    correct = 0
    with torch.no_grad():
        for data, label in test_generator:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print('Accuracy on the test set: %.6s' % (correct / total))

epoch = 5
for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        optimizer.zero_grad() 
        y_pred = model(data)
        loss = criterion(y_pred, label.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss))
    
report_acc()

Net(
  (fc1): Linear(in_features=300, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Epoch:1 Training Loss: 0.861208
Epoch:2 Training Loss: 0.764774
Epoch:3 Training Loss: 0.745584
Epoch:4 Training Loss: 0.726199
Epoch:5 Training Loss: 0.694264
Accuracy on the test set: 0.7080


### 10 features

#### Binary case--m7

In [ ]:
# load dataset

# X_pre_bin10 = np.load("./pretrained/xprebin10.npy")
# y_bin = np.load("./pretrained/ybin.npy")

In [ ]:
# MLP ternary model
import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        hidden_1 = 100
        hidden_2 = 50
        hidden_3 = 2
        self.fc1 = nn.Linear(3000, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return x
    
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y
    

X = torch.from_numpy(X_pre_bin10).float()
y = torch.from_numpy(y_bin).float()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
params = {'batch_size':64, 'num_workers': 0, 'shuffle': True}
train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, **params)
test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)
model = Net()
print(model)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def report_acc():
    total = 0
    correct = 0
    with torch.no_grad():
        for data, label in test_generator:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print('Accuracy on the test set: %.6s' % (correct / total))

epoch = 5
for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        optimizer.zero_grad() 
        y_pred = model(data)
        loss = criterion(y_pred, label.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss))
    
report_acc()

Net(
  (fc1): Linear(in_features=3000, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Epoch:1 Training Loss: 0.498130
Epoch:2 Training Loss: 0.405385
Epoch:3 Training Loss: 0.372602
Epoch:4 Training Loss: 0.347249
Epoch:5 Training Loss: 0.323419
Accuracy on the test set: 0.8312


#### Ternary case--m8

In [ ]:
# load dataset

# X_pre_all10 = np.load("./pretrained/xpreall10.npy")
# y_all = np.load("./pretrained/yall.npy")

In [ ]:
# MLP ternary model
import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        hidden_1 = 100
        hidden_2 = 50
        hidden_3 = 3
        self.fc1 = nn.Linear(3000, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return x
    
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y

X = torch.from_numpy(X_pre_all10).float()
y = torch.from_numpy(y_all).float()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
params = {'batch_size':64, 'num_workers': 0, 'shuffle': True}
train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, **params)
test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)
model = Net()
print(model)
# torch.set_default_tensor_type(torch.DoubleTensor)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

def report_acc():
    total = 0
    correct = 0
    with torch.no_grad():
        for data, label in test_generator:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print('Accuracy on the test set: %.6s' % (correct / total))

epoch = 5
for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        optimizer.zero_grad() 
        y_pred = model(data)
        loss = criterion(y_pred, label.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss))
    
report_acc()

Net(
  (fc1): Linear(in_features=3000, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Epoch:1 Training Loss: 0.836500
Epoch:2 Training Loss: 0.776325
Epoch:3 Training Loss: 0.747137
Epoch:4 Training Loss: 0.722954
Epoch:5 Training Loss: 0.701338
Accuracy on the test set: 0.6517


## Own w2v model

### Average feature

#### Binary case--m9


In [ ]:
# load dataset
# X_bin = np.load("./own/xbin.npy")
# y_bin = np.load("./pretrained/ybin.npy")

In [ ]:
# MLP Binary model

import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        hidden_1 = 100
        hidden_2 = 50
        hidden_3 = 2
        self.fc1 = nn.Linear(300, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return x
    
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y

    
def report_acc():
    total = 0
    correct = 0
    with torch.no_grad():
        for data, label in test_generator:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print('Accuracy on the test set: %.6s' % (correct / total))
    
    
X = torch.from_numpy(X_bin).float()
y = torch.from_numpy(y_bin).float()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
params = {'batch_size':64, 'num_workers': 0, 'shuffle': True}


train_set = Dataset(X_train, y_train)

train_generator = data.DataLoader(train_set, **params)

test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)
    
model = Net()
print(model)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

    
epoch = 5

for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        optimizer.zero_grad() 
        y_pred = model(data)
        loss = criterion(y_pred, label.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss))
    
report_acc()


Net(
  (fc1): Linear(in_features=300, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Epoch:1 Training Loss: 0.446514
Epoch:2 Training Loss: 0.315571
Epoch:3 Training Loss: 0.295449
Epoch:4 Training Loss: 0.285296
Epoch:5 Training Loss: 0.277731
Accuracy on the test set: 0.8856


#### Ternary case--m10


In [ ]:
# load dataset
# X_all = np.load("./own/xall.npy")
# y_all = np.load("./pretrained/yall.npy")

In [ ]:
# MLP ternary model
import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        hidden_1 = 100
        hidden_2 = 50
        hidden_3 = 3
        self.fc1 = nn.Linear(300, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return x
    
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y
    
X = torch.from_numpy(X_all).float()
y = torch.from_numpy(y_all).float()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
params = {'batch_size':64, 'num_workers': 0, 'shuffle': True}
train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, **params)
test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)
model = Net()
print(model)
# torch.set_default_tensor_type(torch.DoubleTensor)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def report_acc():
    total = 0
    correct = 0
    with torch.no_grad():
        for data, label in test_generator:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print('Accuracy on the test set: %.6s' % (correct / total))

epoch = 5
for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        optimizer.zero_grad() 
        y_pred = model(data)
        loss = criterion(y_pred, label.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss))
    
report_acc()

Net(
  (fc1): Linear(in_features=300, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Epoch:1 Training Loss: 0.825166
Epoch:2 Training Loss: 0.680601
Epoch:3 Training Loss: 0.639004
Epoch:4 Training Loss: 0.622262
Epoch:5 Training Loss: 0.612030
Accuracy on the test set: 0.7425


### 10 features

#### Binary case--m11

In [ ]:
# load dataset
# X10_bin = np.load("./own/x10bin.npy")
# y_bin = np.load("./pretrained/ybin.npy")

(200000, 3000)


In [ ]:
# MLP ternary model
import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        hidden_1 = 100
        hidden_2 = 50
        hidden_3 = 2
        self.fc1 = nn.Linear(3000, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return x
    
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y

X = torch.from_numpy(X10_bin).float()
y = torch.from_numpy(y_bin).float()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
params = {'batch_size':64, 'num_workers': 0, 'shuffle': True}
train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, **params)
test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)
model = Net()
print(model)
# torch.set_default_tensor_type(torch.DoubleTensor)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def report_acc():
    total = 0
    correct = 0
    with torch.no_grad():
        for data, label in test_generator:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print('Accuracy on the test set: %.6s' % (correct / total))

epoch = 5
for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        optimizer.zero_grad() 
        y_pred = model(data)
        loss = criterion(y_pred, label.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss))
    
report_acc()

Net(
  (fc1): Linear(in_features=3000, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Epoch:1 Training Loss: 0.405663
Epoch:2 Training Loss: 0.323351
Epoch:3 Training Loss: 0.302963
Epoch:4 Training Loss: 0.284826
Epoch:5 Training Loss: 0.268347
Accuracy on the test set: 0.8594


#### Ternary case--m12


In [ ]:
# load dataset
# X10_all = np.load("./own/x10all.npy")
# y_all = np.load("./pretrained/yall.npy")

In [ ]:
# MLP ternary model
import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        hidden_1 = 100
        hidden_2 = 50
        hidden_3 = 3
        self.fc1 = nn.Linear(3000, hidden_1)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        return x
    
class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y

X = torch.from_numpy(X10_all).float()
y = torch.from_numpy(y_all).float()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
params = {'batch_size':64, 'num_workers': 0, 'shuffle': True}
train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, **params)
test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)
model = Net()
print(model)
# torch.set_default_tensor_type(torch.DoubleTensor)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def report_acc():
    total = 0
    correct = 0
    with torch.no_grad():
        for data, label in test_generator:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    print('Accuracy on the test set: %.6s' % (correct / total))

epoch = 5
for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        optimizer.zero_grad() 
        y_pred = model(data)
        loss = criterion(y_pred, label.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss))
    
report_acc()

Net(
  (fc1): Linear(in_features=3000, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Epoch:1 Training Loss: 0.757325
Epoch:2 Training Loss: 0.648764
Epoch:3 Training Loss: 0.621757
Epoch:4 Training Loss: 0.602401
Epoch:5 Training Loss: 0.583307
Accuracy on the test set: 0.7202


# 5 Recurrent Neueral Network

## RNN tensor preparation

### Pretrained w2v model

In [3]:
import torch
import numpy as np
from torch.autograd import Variable

# compute on gpu
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

cuda


In [ ]:
# RNN tensor
from sklearn.utils import shuffle 



rnn_pre_bin = []
rnn_pre_neu = []

# due to limitation of my computational resource, I cannot handle all the dataset, 
# and thus use 1/5 of it.
df_bin_sh = shuffle(df_bin)
for review in df_bin_sh["review"][:40000]:
#     print(review)
    split_review = review.split(" ")
    len_review = len(split_review)
    try:
        vec = wv[split_review[0]].reshape((1,300))
    except:
        vec = np.zeros(300).reshape((1,300))
    for word in split_review[1:50]:
        try:
            vec = np.vstack((vec, wv[word].reshape((1,300))))
        except KeyError as e:
            continue    
    lines = vec.shape[0]
    if lines < 50:
        vec_zero = np.zeros(300).reshape((1,300))
        for line in range(50 - lines):
            vec = np.vstack((vec, vec_zero))
    rnn_pre_bin.append(vec)
    
    
df_neu_sh = shuffle(df_neu)
for review in df_neu_sh["review"][:10000]:
#     print(review)
    split_review = review.split(" ")
    len_review = len(split_review)
    try:
        vec = wv[split_review[0]].reshape((1,300))
    except:
        vec = np.zeros(300).reshape((1,300))
    for word in split_review[1:50]:
        try:
            vec = np.vstack((vec, wv[word].reshape((1,300))))
        except KeyError as e:
            continue    
    lines = vec.shape[0]
    if lines < 50:
        vec_zero = np.zeros(300).reshape((1,300))
        for line in range(50 - lines):
            vec = np.vstack((vec, vec_zero))
    rnn_pre_neu.append(vec)




Xrnn_pre_bin = np.array(rnn_pre_bin)
# np.save('./pretrained/xrnnprebin.npy',Xrnn_pre_bin)

yrnn_pre_bin = df_bin_sh["label"][:40000].tolist()
yrnn_pre_bin = np.array(yrnn_pre_bin)
# np.save('./pretrained/yrnnprebin.npy',yrnn_pre_bin)

rnn_pre_all = rnn_pre_bin + rnn_pre_neu

Xrnn_pre_all = np.array(rnn_pre_all)
# np.save('./pretrained/xrnnpreall.npy',Xrnn_pre_all)

yrnn_pre_neu = df_neu_sh["label"][:10000].tolist()

yrnn_pre_all = yrnn_pre_bin.tolist() + yrnn_pre_neu
yrnn_pre_all = np.array(yrnn_pre_all)
# np.save('./pretrained/yrnnpreall.npy', yrnn_pre_all)

### Own w2v model

In [12]:
# RNN tensor
from sklearn.utils import shuffle 
import numpy as np
rnn_vec_bin = []
rnn_vec_neu = []

df_bin_sh = shuffle(df_bin)
for review in df_bin_sh["review"][:40000]:
#     print(review)
    split_review = review.split(" ")
    len_review = len(split_review)
    try:
        vec = wvmodel.wv[split_review[0]].reshape((1,300))
    except:
        vec = np.zeros(300).reshape((1,300))
    for word in split_review[1:50]:
        try:
            vec = np.vstack((vec, wvmodel.wv[word].reshape((1,300))))
        except KeyError as e:
            continue    
    lines = vec.shape[0]
    if lines < 50:
        vec_zero = np.zeros(300).reshape((1,300))
        for line in range(50 - lines):
            vec = np.vstack((vec, vec_zero))
    rnn_vec_bin.append(vec)
    
    
df_neu_sh = shuffle(df_neu)
for review in df_neu_sh["review"][:10000]:
#     print(review)
    split_review = review.split(" ")
    len_review = len(split_review)
    try:
        vec = wvmodel.wv[split_review[0]].reshape((1,300))
    except:
        vec = np.zeros(300).reshape((1,300))
    for word in split_review[1:50]:
        try:
            vec = np.vstack((vec, wvmodel.wv[word].reshape((1,300))))
        except KeyError as e:
            continue    
    lines = vec.shape[0]
    if lines < 50:
        vec_zero = np.zeros(300).reshape((1,300))
        for line in range(50 - lines):
            vec = np.vstack((vec, vec_zero))
    rnn_vec_neu.append(vec)




Xrnn_bin = np.array(rnn_vec_bin)
# np.save('./own/xrnnbin.npy',Xrnn_bin)

yrnn_bin = df_bin_sh["label"][:40000].tolist()
yrnn_bin = np.array(yrnn_bin)
# np.save('./own/yrnnbin.npy',yrnn_bin)

rnn_vec_all = rnn_vec_bin + rnn_vec_neu


Xrnn_all = np.array(rnn_vec_all)
# np.save('./own/xrnnall.npy',Xrnn_all)

yrnn_neu = df_neu_sh["label"][:10000].tolist()

yrnn_all = yrnn_bin.tolist() + yrnn_neu
yrnn_all = np.array(yrnn_all)
# np.save('./own/yrnnall.npy', yrnn_all)

## Pretrained w2v model

### RNN

#### Binary case--m13

In [4]:
import numpy as np

In [5]:
# Xrnn_pre_bin = np.load("./pretrained/xrnnprebin.npy")
# yrnn_pre_bin = np.load("./pretrained/yrnnprebin.npy")

In [9]:
# RNN

import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


batch_size = 64
n_hidden = 50
input_size = 300
n_categories = 2
epoch = 60
lr = 0.003

class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y
    
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)
        self.gru = nn.GRU(input_size, hidden_size, batch_first = True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        output, hidden = self.rnn(x, hidden)
        output = self.linear(output[:,-1,:].squeeze(0))
        return output, hidden

def evaluate():
    total = 0
    correct = 0
    hidden_e = None
    with torch.no_grad():
        for data, label in test_generator:
            data = Variable(data).cuda()
            label = Variable(label).cuda()
            y_pred, hidden = rnn(data, hidden_e)
            pred = y_pred.max(1,keepdim=True)[1]

            total += label.size(0)
            correct += pred.eq(label.view_as(pred)).sum().item()
        print('Accuracy on the test set: %.6s' % (correct / total))
    
    

params = {'batch_size': batch_size, 'num_workers': 0, 'shuffle': True}

torch.set_default_tensor_type(torch.DoubleTensor)


X = Xrnn_pre_bin
y = yrnn_pre_bin

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, batch_size = batch_size)

test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)

rnn = RNN(input_size, n_hidden, n_categories).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr=lr)

hidden = torch.zeros(1,batch_size, n_hidden).cuda()


for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        data = Variable(data).cuda()
        label = Variable(label).cuda()
        optimizer.zero_grad() 
        y_pred, hidden = rnn(data, hidden)
        hidden = hidden.detach()
#         label = torch.Tensor(label).long()
        loss = criterion(y_pred, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss), end = " ")
    evaluate()


Epoch:1 Training Loss: 0.695737 Accuracy on the test set: 0.5075
Epoch:2 Training Loss: 0.692722 Accuracy on the test set: 0.5015
Epoch:3 Training Loss: 0.692596 Accuracy on the test set: 0.5008
Epoch:4 Training Loss: 0.692453 Accuracy on the test set: 0.501
Epoch:5 Training Loss: 0.692269 Accuracy on the test set: 0.5672
Epoch:6 Training Loss: 0.692021 Accuracy on the test set: 0.5625
Epoch:7 Training Loss: 0.691666 Accuracy on the test set: 0.5621
Epoch:8 Training Loss: 0.691111 Accuracy on the test set: 0.5603
Epoch:9 Training Loss: 0.690116 Accuracy on the test set: 0.5596
Epoch:10 Training Loss: 0.687813 Accuracy on the test set: 0.5647
Epoch:11 Training Loss: 0.680633 Accuracy on the test set: 0.5805
Epoch:12 Training Loss: 0.674364 Accuracy on the test set: 0.584
Epoch:13 Training Loss: 0.662894 Accuracy on the test set: 0.6598
Epoch:14 Training Loss: 0.618979 Accuracy on the test set: 0.7302
Epoch:15 Training Loss: 0.583634 Accuracy on the test set: 0.6668
Epoch:16 Training Los

#### Ternary case--m14

In [ ]:
# Xrnn_pre_all = np.load("./pretrained/xrnnpreall.npy")
# yrnn_pre_all = np.load("./pretrained/yrnnpreall.npy")

In [ ]:
# RNN

import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

batch_size = 128
n_hidden = 50
input_size = 300
n_categories = 3
epoch = 60
lr = 0.003

class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y
    
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)
        self.gru = nn.GRU(input_size, hidden_size, batch_first = True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        output, hidden = self.rnn(x, hidden)
        output = self.linear(output[:,-1,:].squeeze(0))
        return output, hidden

def evaluate():
    total = 0
    correct = 0
    hidden_e = None
    with torch.no_grad():
        for data, label in test_generator:
            data = Variable(data).cuda()
            label = Variable(label).cuda()
            y_pred, hidden = rnn(data, hidden_e)
            pred = y_pred.max(1,keepdim=True)[1]

            total += label.size(0)
            correct += pred.eq(label.view_as(pred)).sum().item()
        print('Accuracy on the test set: %.6s' % (correct / total))
    
    

params = {'batch_size': batch_size, 'num_workers': 0, 'shuffle': True}

torch.set_default_tensor_type(torch.DoubleTensor)


X = Xrnn_pre_all
y = yrnn_pre_all

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, batch_size = batch_size)

test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)

rnn = RNN(input_size, n_hidden, n_categories).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr=lr)

hidden = torch.zeros(1,batch_size, n_hidden).cuda()


for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        data = Variable(data).cuda()
        label = Variable(label).cuda()
        optimizer.zero_grad() 
        y_pred, hidden = rnn(data, hidden)
        hidden = hidden.detach()
        loss = criterion(y_pred, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}.'.format(e + 1, train_loss), end = " ")
    evaluate()


Epoch:1 Training Loss: 1.064453. Accuracy on the test set: 0.4102
Epoch:2 Training Loss: 1.055470. Accuracy on the test set: 0.4126
Epoch:3 Training Loss: 1.055045. Accuracy on the test set: 0.4134
Epoch:4 Training Loss: 1.054853. Accuracy on the test set: 0.4156
Epoch:5 Training Loss: 1.054681. Accuracy on the test set: 0.4158
Epoch:6 Training Loss: 1.054515. Accuracy on the test set: 0.4172
Epoch:7 Training Loss: 1.054355. Accuracy on the test set: 0.4176
Epoch:8 Training Loss: 1.054199. Accuracy on the test set: 0.418
Epoch:9 Training Loss: 1.054046. Accuracy on the test set: 0.4176
Epoch:10 Training Loss: 1.053895. Accuracy on the test set: 0.417
Epoch:11 Training Loss: 1.053746. Accuracy on the test set: 0.4174
Epoch:12 Training Loss: 1.053598. Accuracy on the test set: 0.4182
Epoch:13 Training Loss: 1.053450. Accuracy on the test set: 0.4188
Epoch:14 Training Loss: 1.053303. Accuracy on the test set: 0.4194
Epoch:15 Training Loss: 1.053156. Accuracy on the test set: 0.4182
Epoch:

### GRU

#### Binary case--m15

In [ ]:
import numpy as np

# Xrnn_pre_bin = np.load("./pretrained/xrnnprebin.npy")
# yrnn_pre_bin = np.load("./pretrained/yrnnprebin.npy")

In [ ]:
import torch


cuda


In [ ]:
# RNN

import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


batch_size = 32
n_hidden = 50
input_size = 300
n_categories = 2
epoch = 50
lr = 0.01

class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y
    
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)
        self.gru = nn.GRU(input_size, hidden_size, batch_first = True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        output = self.linear(output[:,-1,:].squeeze(0))
        return output, hidden

def evaluate():
    total = 0
    correct = 0
    hidden_e = None
    with torch.no_grad():
        for data, label in test_generator:
            data = Variable(data).cuda()
            label = Variable(label).cuda()
            y_pred, hidden = rnn(data, hidden_e)
            pred = y_pred.max(1,keepdim=True)[1]

            total += label.size(0)
            correct += pred.eq(label.view_as(pred)).sum().item()
        print('Accuracy on the test set: %.6s' % (correct / total))
    
    

params = {'batch_size': batch_size, 'num_workers': 0, 'shuffle': True}

torch.set_default_tensor_type(torch.DoubleTensor)


X = Xrnn_pre_bin
y = yrnn_pre_bin



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, batch_size = batch_size)

test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)

rnn = RNN(input_size, n_hidden, n_categories)
rnn.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr=lr)

hidden = torch.zeros(1,batch_size, n_hidden).cuda()


for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        data = Variable(data).cuda()
        label = Variable(label).cuda()
        optimizer.zero_grad() 
        y_pred, hidden = rnn(data, hidden)
        hidden = hidden.detach()
        loss = criterion(y_pred, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}.'.format(e + 1, train_loss), end = " ")
    evaluate()


Epoch:1 Training Loss: 0.693339. Accuracy on the test set: 0.5147
Epoch:2 Training Loss: 0.692461. Accuracy on the test set: 0.5172
Epoch:3 Training Loss: 0.691825. Accuracy on the test set: 0.5205
Epoch:4 Training Loss: 0.691265. Accuracy on the test set: 0.5227
Epoch:5 Training Loss: 0.690765. Accuracy on the test set: 0.5247
Epoch:6 Training Loss: 0.690313. Accuracy on the test set: 0.5247
Epoch:7 Training Loss: 0.689905. Accuracy on the test set: 0.5247
Epoch:8 Training Loss: 0.689532. Accuracy on the test set: 0.5242
Epoch:9 Training Loss: 0.689189. Accuracy on the test set: 0.5255
Epoch:10 Training Loss: 0.688870. Accuracy on the test set: 0.526
Epoch:11 Training Loss: 0.688570. Accuracy on the test set: 0.5267
Epoch:12 Training Loss: 0.688283. Accuracy on the test set: 0.527
Epoch:13 Training Loss: 0.688005. Accuracy on the test set: 0.5272
Epoch:14 Training Loss: 0.687731. Accuracy on the test set: 0.5277
Epoch:15 Training Loss: 0.687458. Accuracy on the test set: 0.5282
Epoch:

#### Ternary case--m16

In [ ]:
# Xrnn_pre_all = np.load("./pretrained/xrnnpreall.npy")
# yrnn_pre_all = np.load("./pretrained/yrnnpreall.npy")

In [ ]:
# RNN

import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

batch_size = 32
n_hidden = 50
input_size = 300
n_categories = 3
epoch = 50
lr = 0.01

class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y
    
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)
        self.gru = nn.GRU(input_size, hidden_size, batch_first = True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        output = self.linear(output[:,-1,:].squeeze(0))
        return output, hidden

def evaluate():
    total = 0
    correct = 0
    hidden_e = None
    with torch.no_grad():
        for data, label in test_generator:
            data = Variable(data).cuda()
            label = Variable(label).cuda()
            y_pred, hidden = rnn(data, hidden_e)
            pred = y_pred.max(1,keepdim=True)[1]

            total += label.size(0)
            correct += pred.eq(label.view_as(pred)).sum().item()
        print('Accuracy on the test set: %.6s' % (correct / total))
    
    

params = {'batch_size': batch_size, 'num_workers': 0, 'shuffle': True}

torch.set_default_tensor_type(torch.DoubleTensor)


X = Xrnn_pre_all
y = yrnn_pre_all

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, batch_size = batch_size)

test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)

rnn = RNN(input_size, n_hidden, n_categories)
rnn.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr=lr)

hidden = torch.zeros(1,batch_size, n_hidden).cuda()


for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        data = Variable(data).cuda()
        label = Variable(label).cuda()
        optimizer.zero_grad() 
        y_pred, hidden = rnn(data, hidden)
        hidden = hidden.detach()
#         label = torch.Tensor(label).long()
        loss = criterion(y_pred, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}.'.format(e + 1, train_loss), end = " ")
    evaluate()


Epoch:1 Training Loss: 1.061493. Accuracy on the test set: 0.4154
Epoch:2 Training Loss: 1.054421. Accuracy on the test set: 0.4224
Epoch:3 Training Loss: 1.053701. Accuracy on the test set: 0.4266
Epoch:4 Training Loss: 1.053140. Accuracy on the test set: 0.428
Epoch:5 Training Loss: 1.052667. Accuracy on the test set: 0.4286
Epoch:6 Training Loss: 1.052259. Accuracy on the test set: 0.4292
Epoch:7 Training Loss: 1.051900. Accuracy on the test set: 0.4294
Epoch:8 Training Loss: 1.051581. Accuracy on the test set: 0.4292
Epoch:9 Training Loss: 1.051290. Accuracy on the test set: 0.4296
Epoch:10 Training Loss: 1.051020. Accuracy on the test set: 0.4298
Epoch:11 Training Loss: 1.050766. Accuracy on the test set: 0.43
Epoch:12 Training Loss: 1.050521. Accuracy on the test set: 0.431
Epoch:13 Training Loss: 1.050280. Accuracy on the test set: 0.4314
Epoch:14 Training Loss: 1.050041. Accuracy on the test set: 0.431
Epoch:15 Training Loss: 1.049801. Accuracy on the test set: 0.4308
Epoch:16 

# Own w2v model

### RNN

#### Binary case--m17

In [7]:
# Xrnn_bin = np.load("./own/xrnnbin.npy")
# yrnn_bin = np.load("./own/yrnnbin.npy")

In [12]:
# RNN

import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

batch_size = 128
n_hidden = 50
input_size = 300
n_categories = 2
epoch = 80
lr = 0.003

class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y
    
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)
        self.gru = nn.GRU(input_size, hidden_size, batch_first = True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        output, hidden = self.rnn(x, hidden)
        output = self.linear(output[:,-1,:].squeeze(0))
        return output, hidden

def evaluate():
    total = 0
    correct = 0
    hidden_e = None
    with torch.no_grad():
        for data, label in test_generator:
            data = Variable(data).cuda()
            label = Variable(label).cuda()
            y_pred, hidden = rnn(data, hidden_e)
            pred = y_pred.max(1,keepdim=True)[1]

            total += label.size(0)
            correct += pred.eq(label.view_as(pred)).sum().item()
        print('Accuracy on the test set: %.6s' % (correct / total))
    
    

params = {'batch_size': batch_size, 'num_workers': 0, 'shuffle': True}

torch.set_default_tensor_type(torch.DoubleTensor)


X = Xrnn_bin
y = yrnn_bin

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, batch_size = batch_size)

test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)

rnn = RNN(input_size, n_hidden, n_categories).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr=lr)

hidden = torch.zeros(1,batch_size, n_hidden).cuda()


for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        data = Variable(data).cuda()
        label = Variable(label).cuda()
        optimizer.zero_grad() 
        y_pred, hidden = rnn(data, hidden)
        hidden = hidden.detach()
#         label = torch.Tensor(label).long()
        loss = criterion(y_pred, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss), end = " ")
    evaluate()


Epoch:1 Training Loss: 0.694210 Accuracy on the test set: 0.5258
Epoch:2 Training Loss: 0.693704 Accuracy on the test set: 0.5291
Epoch:3 Training Loss: 0.693312 Accuracy on the test set: 0.5308
Epoch:4 Training Loss: 0.692932 Accuracy on the test set: 0.5313
Epoch:5 Training Loss: 0.692540 Accuracy on the test set: 0.534
Epoch:6 Training Loss: 0.692123 Accuracy on the test set: 0.5352
Epoch:7 Training Loss: 0.691669 Accuracy on the test set: 0.5365
Epoch:8 Training Loss: 0.691167 Accuracy on the test set: 0.5395
Epoch:9 Training Loss: 0.690605 Accuracy on the test set: 0.542
Epoch:10 Training Loss: 0.689970 Accuracy on the test set: 0.543
Epoch:11 Training Loss: 0.689250 Accuracy on the test set: 0.5438
Epoch:12 Training Loss: 0.688437 Accuracy on the test set: 0.5452
Epoch:13 Training Loss: 0.687528 Accuracy on the test set: 0.5452
Epoch:14 Training Loss: 0.686529 Accuracy on the test set: 0.5475
Epoch:15 Training Loss: 0.685462 Accuracy on the test set: 0.5515
Epoch:16 Training Loss

#### Ternary case--m18

In [5]:
# Xrnn_all = np.load("./own/xrnnall.npy")
# yrnn_all = np.load("./own/yrnnall.npy")

In [8]:
# RNN

import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

batch_size = 64
n_hidden = 50
input_size = 300
n_categories = 3
epoch = 60
lr = 0.002

class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y
    
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)
        self.gru = nn.GRU(input_size, hidden_size, batch_first = True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        output, hidden = self.rnn(x, hidden)
        output = self.linear(output[:,-1,:].squeeze(0))
        return output, hidden

def evaluate():
    total = 0
    correct = 0
    hidden_e = None
    with torch.no_grad():
        for data, label in test_generator:
            data = Variable(data).cuda()
            label = Variable(label).cuda()
            y_pred, hidden = rnn(data, hidden_e)
            pred = y_pred.max(1,keepdim=True)[1]

            total += label.size(0)
            correct += pred.eq(label.view_as(pred)).sum().item()
        print('Accuracy on the test set: %.6s' % (correct / total))
    
    

params = {'batch_size': batch_size, 'num_workers': 0, 'shuffle': True}

torch.set_default_tensor_type(torch.DoubleTensor)


X = Xrnn_all
y = yrnn_all

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, batch_size = batch_size)

test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)

rnn = RNN(input_size, n_hidden, n_categories).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr=lr)

hidden = torch.zeros(1,batch_size, n_hidden).cuda()


for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        data = Variable(data).cuda()
        label = Variable(label).cuda()
        optimizer.zero_grad() 
        y_pred, hidden = rnn(data, hidden)
        hidden = hidden.detach()
#         label = torch.Tensor(label).long()
        loss = criterion(y_pred, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss), end = " ")
    evaluate()


Epoch:1 Training Loss: 1.076744 Accuracy on the test set: 0.4252
Epoch:2 Training Loss: 1.057025 Accuracy on the test set: 0.4332
Epoch:3 Training Loss: 1.055142 Accuracy on the test set: 0.4369
Epoch:4 Training Loss: 1.053933 Accuracy on the test set: 0.4403
Epoch:5 Training Loss: 1.052693 Accuracy on the test set: 0.4428
Epoch:6 Training Loss: 1.051418 Accuracy on the test set: 0.4455
Epoch:7 Training Loss: 1.050142 Accuracy on the test set: 0.4513
Epoch:8 Training Loss: 1.048922 Accuracy on the test set: 0.4536
Epoch:9 Training Loss: 1.047815 Accuracy on the test set: 0.454
Epoch:10 Training Loss: 1.046859 Accuracy on the test set: 0.4572
Epoch:11 Training Loss: 1.046049 Accuracy on the test set: 0.4582
Epoch:12 Training Loss: 1.045348 Accuracy on the test set: 0.4596
Epoch:13 Training Loss: 1.044710 Accuracy on the test set: 0.4597
Epoch:14 Training Loss: 1.044087 Accuracy on the test set: 0.46
Epoch:15 Training Loss: 1.043439 Accuracy on the test set: 0.4626
Epoch:16 Training Loss

### GRU

#### Binary case--m19

In [ ]:
# RNN

import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

batch_size = 32
n_hidden = 50
input_size = 300
n_categories = 2
epoch = 20
lr = 0.1

class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y
    
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)
        self.gru = nn.GRU(input_size, hidden_size, batch_first = True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        output = self.linear(output[:,-1,:].squeeze(0))
        return output, hidden

def evaluate():
    total = 0
    correct = 0
    hidden_e = None
    with torch.no_grad():
        for data, label in test_generator:
            data = Variable(data).cuda()
            label = Variable(label).cuda()  
            y_pred, hidden = rnn(data, hidden_e)
            pred = y_pred.max(1,keepdim=True)[1]

            total += label.size(0)
            correct += pred.eq(label.view_as(pred)).sum().item()
        print('Accuracy on the test set: %.6s' % (correct / total))
    
    

params = {'batch_size': batch_size, 'num_workers': 0, 'shuffle': True}

torch.set_default_tensor_type(torch.DoubleTensor)


X = Xrnn_bin
y = yrnn_bin

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, batch_size = batch_size)

test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)

rnn = RNN(input_size, n_hidden, n_categories).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr=lr)

hidden = torch.zeros(1,batch_size, n_hidden).cuda()


for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        data = Variable(data).cuda()
        label = Variable(label).cuda()
        optimizer.zero_grad() 
        y_pred, hidden = rnn(data, hidden)
        hidden = hidden.detach()
#         label = torch.Tensor(label).long()
        loss = criterion(y_pred, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss), end = " ")
    evaluate()


Epoch:1 Training Loss: 0.688657 Accuracy on the test set: 0.547
Epoch:2 Training Loss: 0.678230 Accuracy on the test set: 0.5682
Epoch:3 Training Loss: 0.666195 Accuracy on the test set: 0.5707
Epoch:4 Training Loss: 0.670846 Accuracy on the test set: 0.5152
Epoch:5 Training Loss: 0.708411 Accuracy on the test set: 0.5215
Epoch:6 Training Loss: 0.650411 Accuracy on the test set: 0.7315
Epoch:7 Training Loss: 0.476969 Accuracy on the test set: 0.8215
Epoch:8 Training Loss: 0.383537 Accuracy on the test set: 0.844
Epoch:9 Training Loss: 0.339800 Accuracy on the test set: 0.8425
Epoch:10 Training Loss: 0.310801 Accuracy on the test set: 0.8382
Epoch:11 Training Loss: 0.289215 Accuracy on the test set: 0.8442
Epoch:12 Training Loss: 0.271746 Accuracy on the test set: 0.8422
Epoch:13 Training Loss: 0.252675 Accuracy on the test set: 0.843
Epoch:14 Training Loss: 0.234210 Accuracy on the test set: 0.84
Epoch:15 Training Loss: 0.217016 Accuracy on the test set: 0.8355
Epoch:16 Training Loss: 

#### Ternary case--m20

In [ ]:
# Xrnn_all = np.load("./own/xrnnall.npy")
# yrnn_all = np.load("./own/yrnnall.npy")

In [ ]:
# RNN

import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

batch_size = 32
n_hidden = 50
input_size = 300
n_categories = 3
epoch = 40
lr = 0.1

class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.label = y
        self.data = X
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        X = self.data[index]
        y = self.label[index]
    
        return X, y
    
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.rnn = nn.RNN(input_size, hidden_size, batch_first = True)
        self.gru = nn.GRU(input_size, hidden_size, batch_first = True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        output = self.linear(output[:,-1,:].squeeze(0))
        return output, hidden

def evaluate():
    total = 0
    correct = 0
    hidden_e = None
    with torch.no_grad():
        for data, label in test_generator:
            data = Variable(data).cuda()
            label = Variable(label).cuda()
            y_pred, hidden = rnn(data, hidden_e)
            pred = y_pred.max(1,keepdim=True)[1]

            total += label.size(0)
            correct += pred.eq(label.view_as(pred)).sum().item()
        print('Accuracy on the test set: %.6s' % (correct / total))
    
    

params = {'batch_size': batch_size, 'num_workers': 0, 'shuffle': True}

torch.set_default_tensor_type(torch.DoubleTensor)


X = Xrnn_all
y = yrnn_all

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_set = Dataset(X_train, y_train)
train_generator = data.DataLoader(train_set, batch_size = batch_size)

test_set = Dataset(X_test, y_test)
test_generator = data.DataLoader(test_set, **params)

rnn = RNN(input_size, n_hidden, n_categories).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr=lr)

hidden = torch.zeros(1,batch_size, n_hidden).cuda()


for e in range(epoch):
    train_loss = 0.0
    for data, label in train_generator:
        data = Variable(data).cuda()
        label = Variable(label).cuda()
        optimizer.zero_grad() 
        y_pred, hidden = rnn(data, hidden)
        hidden = hidden.detach()
#         label = torch.Tensor(label).long()
        loss = criterion(y_pred, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    train_loss = train_loss / len(train_generator.dataset)
    print('Epoch:{} Training Loss: {:.6f}'.format(e + 1, train_loss), end = " ")
    evaluate()


Epoch:1 Training Loss: 1.051962 Accuracy on the test set: 0.442
Epoch:2 Training Loss: 1.041174 Accuracy on the test set: 0.4776
Epoch:3 Training Loss: 1.031727 Accuracy on the test set: 0.4132
Epoch:4 Training Loss: 0.987037 Accuracy on the test set: 0.5622
Epoch:5 Training Loss: 0.987023 Accuracy on the test set: 0.5234
Epoch:6 Training Loss: 0.986022 Accuracy on the test set: 0.4008
Epoch:7 Training Loss: 1.007021 Accuracy on the test set: 0.4
Epoch:8 Training Loss: 1.034489 Accuracy on the test set: 0.4056
Epoch:9 Training Loss: 0.945747 Accuracy on the test set: 0.6342
Epoch:10 Training Loss: 0.807497 Accuracy on the test set: 0.6626
Epoch:11 Training Loss: 0.750493 Accuracy on the test set: 0.6842
Epoch:12 Training Loss: 0.705669 Accuracy on the test set: 0.6902
Epoch:13 Training Loss: 0.669770 Accuracy on the test set: 0.6838
Epoch:14 Training Loss: 0.639800 Accuracy on the test set: 0.6766
Epoch:15 Training Loss: 0.611865 Accuracy on the test set: 0.6698
Epoch:16 Training Loss: